Before we move on to other data retrieval activities, it is useful to explore what we found in consulting ITIS with either declared TSN identifiers or scientific name searches to see if there is any further work that can be done to improve our ability and accuracy in getting other data sources pulled together. This notebook examines the cached ITIS results and makes some decisions about what to do next.

In [1]:
import json
import bispy
import jsonschema
from collections import Counter

bis_utils = bispy.bis.Utils()

import helperfunctions

In [2]:
with open("../cache/workplan_species.json", "r") as f:
    workplan_species = json.loads(f.read())
    f.close()

with open("../cache/itis.json", "r") as f:
    itis_cache = json.loads(f.read())
    f.close()

All of the cases where we returned more than one ITIS record from the function mean that something interesting happened. We can check this pretty quickly by looking at the "Processing Metadata" that our function records. In all of the cases here, our code followed the accepted TSN declared at the point of discovery, whether based on what appears to have been an invalid TSN assignment from the FWS source information or a name lookup that was invalid but for which there is a valid record in ITIS.

The following codeblock lets us examine what is going on in these cases. The Processing Metadata structure is what our function records about what it did. It includes the URLs to the ITIS API that resulted in some action. We record both the valid/accepted and invalid/unaccepted names from ITIS and reach back to the workplan_species to show that record.

In [3]:
for r in [i for i in itis_cache if "itis_data" in i.keys() and len(i["itis_data"]) > 1]:
    source_url = next((o for o in r["processing_metadata"]["details"] if "Exact Match" in o.keys()), None)
    if not source_url:
        source_url = next((o for o in r["processing_metadata"]["details"] if "Exact Match Fail" in o.keys()), None)
    source_url = next((v for k, v in source_url.items()), None)
    source_identifier = source_url.split(":")[-1].replace("\%20", " ")
    if source_identifier.isdigit():
        print("ITIS TSN used in lookup:", source_identifier)
    else:
        print("Scientific name used in lookup:", source_identifier)
    
    display(r["processing_metadata"])


ITIS TSN used in lookup: 207135


{'status': 'success',
 'date_processed': '2019-07-24T15:48:35.628743',
 'status_message': 'Followed Accepted TSN',
 'details': [{'TSN Search': 'http://services.itis.gov/?wt=json&rows=10&q=tsn:773525'},
  {'Exact Match': 'http://services.itis.gov/?wt=json&rows=10&q=tsn:207135'}]}

ITIS TSN used in lookup: 80066


{'status': 'success',
 'date_processed': '2019-07-24T15:48:35.628743',
 'status_message': 'Followed Accepted TSN',
 'details': [{'TSN Search': 'http://services.itis.gov/?wt=json&rows=10&q=tsn:983630'},
  {'Exact Match': 'http://services.itis.gov/?wt=json&rows=10&q=tsn:80066'}]}

ITIS TSN used in lookup: 80079


...

{'status': 'success',
 'date_processed': '2019-07-24T15:48:35.628743',
 'status_message': 'Followed Accepted TSN',
 'details': [{'TSN Search': 'http://services.itis.gov/?wt=json&rows=10&q=tsn:898420'},
  {'Exact Match': 'http://services.itis.gov/?wt=json&rows=10&q=tsn:898971'}]}

ITIS TSN used in lookup: 174277


{'status': 'success',
 'date_processed': '2019-07-24T15:48:35.628743',
 'status_message': 'Followed Accepted TSN',
 'details': [{'TSN Search': 'http://services.itis.gov/?wt=json&rows=10&q=tsn:1082386'},
  {'Exact Match': 'http://services.itis.gov/?wt=json&rows=10&q=tsn:174277'}]}

ITIS TSN used in lookup: 174277


{'status': 'success',
 'date_processed': '2019-07-24T15:48:35.628743',
 'status_message': 'Followed Accepted TSN',
 'details': [{'TSN Search': 'http://services.itis.gov/?wt=json&rows=10&q=tsn:1082386'},
  {'Exact Match': 'http://services.itis.gov/?wt=json&rows=10&q=tsn:174277'}]}

There are essentially three things that seem to be going on here:

1. There are cases where the FWS data declares a TSN for what ITIS considers to be invalid (animals)/unaccepted (plants) taxonomy. It could be that FWS biologists disagree with ITIS or simply a matter of FWS information being out of date. We don't have enough information as yet to make a judgment.
2. There are cases where the FWS data uses a scientific name that ITIS considers to be invalid/unaccepted. This is kind of the same issue as no. 1, but it could be that FWS is out of date with whatever has happened in the ITIS taxonomy world.
3. There is one case of a misspelling (Macroclemys temmincki should have been Macroclemys temminckii) where the search had to go to a fuzzy match to find the record. In this case, ITIS considers one of its records to be invalid and the other valid, though they are ultimately for the same name. 

Because there could be disagreement on the part of species biologists with the taxonomic authority, the conservative course of action here is to record the valid ITIS name and other information as an additional potential point of reference. If we run into cases where the FWS name does not find results in another system we are checking for data, we can try the ITIS valid/accepted name to see what we can turn up, flagging that result with some uncertainty factor or note to help determine its utility.

The following code block establishes a connection back to the appropriate originating record in the workplan_species data and injects ITIS valid/accepted names into a new attribute, just for these cases where we may want to use that additional information.

In [4]:
updated_workplan_species = list()

for r in [i for i in itis_cache if "itis_data" in i.keys() and len(i["itis_data"]) > 1]:
    source_url = next((o for o in r["processing_metadata"]["details"] if "Exact Match" in o.keys()), None)
    if not source_url:
        source_url = next((o for o in r["processing_metadata"]["details"] if "Exact Match Fail" in o.keys()), None)
    source_url = next((v for k, v in source_url.items()), None)
    source_identifier = source_url.split(":")[-1].replace("\%20", " ")
    if source_identifier.isdigit():
        source_workplan_species = next((s for s in workplan_species if s["ITIS TSN"] == source_identifier), None)
    else:
        source_workplan_species = next((s for s in workplan_species if s["Lookup Name"] == source_identifier), None)
    
    source_workplan_species["Valid ITIS Scientific Name"] = next((i["nameWOInd"] for i in r["itis_data"] if i["usage"] in ["valid","accepted"]), None)
    source_workplan_species["Valid ITIS TSN"] = next((i["tsn"] for i in r["itis_data"] if i["usage"] in ["valid","accepted"]), None)
    updated_workplan_species.append(source_workplan_species)


In [5]:
# Build a new workplan species recordset, injecting the augmented records
new_workplan_species = list()
for s in workplan_species:
    new_record = next((r for r in updated_workplan_species if r["Lookup Name"] == s["Lookup Name"]), None)
    if new_record:
        new_workplan_species.append(new_record)
    else:
        new_workplan_species.append(s)

# Cache the new set of workplan species information
display(bis_utils.doc_cache("../cache/workplan_species.json", new_workplan_species))

{'Doc Cache File': 'cache/workplan_species.json',
 'Number of Documents in Cache': 373,
 'Document Number 39': {'Guild': 'Bivalves',
  'Species Name (Common)': 'triangle pigtoe',
  'Scientific Name': 'Fusconaia lananensis',
  'Lead FWS Regional Office': 'Region 2 - Southwest',
  'Proposed FWS Decision Timeframe (Fiscal Year)': 2019,
  'Range': 'TX',
  'Bin': None,
  'ECOS Link': 'https://ecos.fws.gov/ecp/species/10339',
  'ITIS TSN': '80048',
  'Lookup Name': 'Fusconaia lananensis'}}

# Schema Validation
Building out the schema for the workplan_species dataset used in subsequent lookup and data assembly steps gave me an opportunity to explore the enum aspect of the JSON Schema specification. This is a way of forcing a set of available values for a property, analogous to specifying enumerated domain values in FGDC metadata or ISO Feature Catalog. There were a few properties in the workplan species list that lend themselves to this approach. As we look toward scaling this methodology, we will develop the semantic infrastructure behind the system such that defined "code lists" and more robust vocabularies are used to drive validation mechanisms.

I also ran into a technical snag in using the jsonschema package for validation where passing the entire list of workplan species documents resulted in a recursion error that I could not resolve. Alternatively, I iterated through each document to validate against the schema individually, which was not noticeably slower anyway.

In [6]:
workplan_species_schema = helperfunctions.load_schema('workplan_species')
display(workplan_species_schema)

for record in new_workplan_species:
    jsonschema.validate(record, workplan_species_schema)

{'definitions': {'items': {'$id': '#/items',
   'type': ['object', 'array'],
   'title': 'Generic container for items in a dataset',
   'description': 'A JSON array or object property containing one or more items in a dataset'},
  'doi': {'$id': '#doi',
   'type': ['string', 'null'],
   'title': 'Digital Object Identifier',
   'description': 'A digital object identifier for or associated with a record. May be in the form of an HTTP url or a standalone identifier.',
   'examples': ['http://dx.doi.org/10.2305/IUCN.UK.2004.RLTS.T59435A11941314.en',
    '10.2305/IUCN.UK.2004.RLTS.T59435A11941314.en']},
  'resolvable_identifier': {'$id': '#resolvable_identifier',
   'type': 'string',
   'title': 'Resolvable Identifier',
   'description': 'Some form of resolvable identifier for a record that returns a response when accessed over an included protocol such as HTTP. May or may not provide for content negotiation.',
   'examples': ['https://www.iucnredlist.org/species/59435/11941314']},
  'citat